In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# data loading

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')
labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
train.head()

In [ ]:
len(train['subject'].unique())

In [ ]:
plt.hist(train.groupby('subject').count()['sequence'])
plt.xlabel('number of data')
plt.ylabel('number of subject')
plt.show

# Visualized 10 data for each sensor for each state

In [ ]:
column_name = train.columns
sq_No_state0_10 = labels[labels['state']==0].head(10)['sequence'].reset_index(drop=True)
sq_No_state1_10 = labels[labels['state']==1].head(10)['sequence'].reset_index(drop=True)

In [ ]:
for i in range(3,16):
  fig1 = plt.figure(figsize=(20, 5))
  fig1.suptitle(f'{column_name[i]}, state:0', fontsize=16)
  plt.subplots_adjust(wspace=0.4, hspace=0.3)
  for j in range(len(sq_No_state0_10)):
    plt.subplot(1, 10, j+1)
    temp = train[train['sequence']==sq_No_state0_10[j]]
    plt.plot(temp['step'], temp[column_name[i]])
    plt.title(f'sequence_No:{sq_No_state0_10[j]}')
    plt.xlabel('seconds')
    plt.ylabel('signal')
    plt.ylim(-10, 10)

  fig2 = plt.figure(figsize=(20, 5))
  fig2.suptitle(f'{column_name[i]}, state:1', fontsize=16)
  plt.subplots_adjust(wspace=0.5, hspace=0.2)
  for j in range(len(sq_No_state1_10)):
    plt.subplot(1, 10, j+1)
    temp = train[train['sequence']==sq_No_state1_10[j]]
    plt.plot(temp['step'], temp[column_name[i]], color='forestgreen')
    plt.title(f'sequence_No:{sq_No_state1_10[j]}')
    plt.xlabel('seconds')
    plt.ylabel('signal')
    plt.ylim(-10, 10)
plt.show()